In [1]:
import pandas as pd
import json
import os
import re

In [2]:
def File_merge_json(Folder):  #Folder文件夹路径
    try:
        File_List = []  #存储文件夹下所有文件的路径
        DataFrame_List = []  #存储dataframe
        for root,dirs,files in os.walk(Folder):
            for file in files:
                file_with_path = os.path.join(root,file)
                File_List.append(file_with_path)
                data_original = pd.read_json(file_with_path,encoding="utf-8")
                DataFrame_List.append(data_original)
        datas = DataFrame_List
        return datas
    except  Exception as e: 
        print(e)
dataAll = File_merge_json(r"G:\GitWorkPlace\WorkPlace\PythonWorkPlace\spider\spidertest\抖音电商罗盘spider\抖店爬虫\电信100")
area=pd.read_excel(r"./省份.xlsx")

In [3]:
# dataAll[0]["data"][0][1]["product_info"]

In [4]:
need_data = pd.DataFrame(columns=["店铺名称","成交金额_min","成交金额_max","点击次数_min","点击次数_max","商品ID","商品名称","是否首次上榜","价格区间","排名"])
for a in dataAll:
    datasss = pd.DataFrame(columns=["店铺名称","成交金额_min","成交金额_max","点击次数_min","点击次数_max","商品ID","商品名称","是否首次上榜","价格区间","排名"])
    lsits = a["data"][0]
    for i in lsits:
        info_dicts = {}
        info_dicts["店铺名称"] = i["product_info"]["shop_list"][0]["shop_name"]
        info_dicts["商品名称"] = i["product_info"]["name"]
        info_dicts["商品ID"] = i["product_info"]["id"]
        # info_dicts["排名变化"] = i["product_info"]["rank_change"]
        try:
            info_dicts["点击次数_min"] = i["product_click_cnt"]["value_range"][0]["value"]
            info_dicts["点击次数_max"] = i["product_click_cnt"]["value_range"][1]["value"]
        except Exception as e:
            # print(e)
            pass
        finally:
            try:
                info_dicts["成交金额_min"] = i["new_pay_amt"]["value_range"][0]["value"]
                info_dicts["成交金额_max"] = i["new_pay_amt"]["value_range"][1]["value"]
            except Exception as e:
                # print(e)
                pass   
            finally:    
                info_dicts["价格区间"] = i["product_info"]["price_bin"]
                info_dicts["是否首次上榜"] = i["product_info"]["newly_on_ranking"]
                info_dicts["排名"] = i["product_info"]["rank"]
                info_df = pd.DataFrame(info_dicts,index=[0])
                datasss = pd.concat([datasss,info_df])
    need_data = pd.concat([need_data,datasss])
    


In [5]:
#数据清洗
need_data["成交金额_max"] = need_data["成交金额_max"]/100
need_data["成交金额_min"] = need_data["成交金额_min"]/100
need_data.sort_values(by="排名",inplace=True)

In [6]:
# def get_data(a):
#     data = pd.DataFrame(columns=["店铺名称","成交金额_min","成交金额_max","点击次数_min","点击次数_max","商品ID","商品名称","是否首次上榜","价格区间","排名"])
#     lsits = a["data"][0]
#     for i in lsits:
#         info_dicts = {}
#         info_dicts["店铺名称"] = i["product_info"]["shop_list"][0]["shop_name"]
#         info_dicts["商品名称"] = i["product_info"]["name"]
#         info_dicts["商品ID"] = i["product_info"]["id"]
#         info_dicts["点击次数_min"] = i["product_click_cnt"]["value_range"]
#         info_dicts["点击次数_max"] = i["product_click_cnt"]["value_range"]
#         info_dicts["成交金额_min"] = i["new_pay_amt"]["value_range"][0]["value"]
#         info_dicts["成交金额_max"] = i["new_pay_amt"]["value_range"][1]["value"]
#         info_dicts["价格区间"] = i["product_info"]["price_bin"]
#         info_dicts["是否首次上榜"] = i["product_info"]["newly_on_ranking"]
#         info_dicts["排名"] = i["product_info"]["rank"]
#         info_df = pd.DataFrame(info_dicts,index=[0])
#         data = pd.concat([data,info_df])
#     return data

In [7]:
# need_data = pd.DataFrame(columns=["店铺名称","成交金额_min","成交金额_max","点击次数_min","点击次数_max","商品ID","商品名称","价格区间","排名"])
# for data in dataAll:
#     A = get_data(data)
#     need_data = pd.concat([need_data,A])
# need_data.to_excel(".\spider数据.xlsx",index=False)

In [8]:
#提取省份
def test(x):
    try:
        x["省份"]=[sa_name for sa_name in area["sa_name"] if re.search(sa_name,x["商品名称"])] [0]
    except Exception as e:
            print(e)
    finally:
        return x
need_data =need_data.apply(test,axis=1)

list index out of range


In [9]:
need_data.head(1)

,价格区间,商品ID,商品名称,店铺名称,成交金额_max,成交金额_min,排名,是否首次上榜,点击次数_max,点击次数_min,省份
0,¥100-¥100,3462283693555613740,安徽电信100元手机话费充值,千行你我充值专营店,500000.0,250000.0,1,False,100,50,安徽


In [10]:
need_data.to_excel(".\spider数据.xlsx",index=False)